[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seagarwu/mynotebooks/blob/main/shioaji/shioaji_onlinedoc/shioaji-market_data-historical_market_data.ipynb)

In [ ]:
import shioaji as sj
import getpass as gp

api = sj.Shioaji()
user_id, user_pass = gp.getpass("USER ID: "), gp.getpass("USER PASS: ")
accounts = api.login(user_id, user_pass)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


# Historical Market Data

## Ticks Data

### Ticks Schema

In [ ]:
from shioaji.data import Ticks
Ticks??

Init signature:
Ticks(
    *,
    ts: List[int],
    close: List[float],
    volume: List[int],
    bid_price: List[float],
    bid_volume: List[int],
    ask_price: List[float],
    ask_volume: List[int],
    tick_type: List[int],
) -> None
Source:        
class Ticks(BaseModel):
    ts: typing.List[int]
    close: typing.List[float]
    volume: typing.List[int]
    bid_price: typing.List[float]
    bid_volume: typing.List[int]
    ask_price: typing.List[float]
    ask_volume: typing.List[int]
    tick_type: typing.List[int]

    def lazy_setter(self, **kwargs):
        [
            setattr(self, kwarg, value)
            for kwarg, value in kwargs.items()
            if hasattr(self, kwarg)
        ]
File:           c:\users\alan\anaconda3\envs\python3.7\lib\site-packages\shioaji\data.py
Type:           ModelMetaclass
Subclasses:     


所回傳的資料為shioaji.data.Ticks物件，屬性說明如下：

|屬性|值|說明|
|:----|:----|:----|
|ts|[1631889000000000000]|時間戳|
|close|[600.0]|收盤價，即成交價格|
|volume|[57]|成交量|
|bid_price|[600.0]|委買價(買進)|
|bid_volume|[1107]|委買量|
|ask_price|[601.0]|委賣價(賣出)|
|ask_volume|[3]|委賣量|

跟之前抓到的資料不同的地方在於，所回傳的資料是一個shioaji.data.Ticks物件而非List，也就是把所有的tick資料存在一個物件中，而物件中的屬性內容為List，用來放每一個tick的資料。

### Get ticks all day

In [ ]:
# 抓指定日期的tick資料 -> 這個很好用
ticks = api.ticks(
    contract=api.Contracts.Stocks["2330"], 
    date="2020-03-04"
)
ticks

In [ ]:
# 抓指定日期的tick資料 -> 這個很好用
ticks = api.ticks(
    contract=api.Contracts.Futures.MXF.MXF202205,
    date="2022-04-29"
)
ticks

### Ticks to DataFrame
使用api.ticks取得資料，若未經過處理，在閱讀上較為困難，此時可以先把tick資料轉換為DataFrame。但因為資料內容格式與之前的格式不同，所以在轉換前要先將物件內容先轉換為Dict。

In [ ]:
import pandas as pd
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df.head()

,close,ask_volume,volume,bid_price,bid_volume,ts,tick_type,ask_price
0,322.0,646,5098,321.5,5,2020-03-04 09:00:00.821,0,322.0
1,321.5,13,91,321.0,100,2020-03-04 09:00:05.836,0,321.5
2,321.0,31,126,321.0,94,2020-03-04 09:00:10.849,2,321.5
3,321.0,86,59,321.0,78,2020-03-04 09:00:15.864,2,321.5
4,321.0,199,90,321.0,20,2020-03-04 09:00:20.877,2,321.5


### Get ticks range time

In [ ]:
# 抓指定日期中特定時段的tick資料 -> 這個很好用。看起來如果要做秒K線的話，要寫迴圈抓每一天的tick資料
ticks = api.ticks(
    contract=api.Contracts.Stocks["2330"], 
    date="2022-04-26", 
    query_type=sj.constant.TicksQueryType.RangeTime,
    time_start="09:00:00",
    time_end="09:20:01"
)
ticks

In [ ]:
import pandas as pd
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df.head()

,close,ask_volume,volume,bid_price,bid_volume,ts,tick_type,ask_price
0,550.0,218,3086,550.0,407,2022-04-26 09:00:03.006512,1,551.0
1,550.0,509,15,550.0,469,2022-04-26 09:00:03.147527,2,551.0
2,551.0,508,1,550.0,469,2022-04-26 09:00:03.167291,1,551.0
3,550.0,508,105,550.0,364,2022-04-26 09:00:03.172808,2,551.0
4,551.0,516,1,550.0,365,2022-04-26 09:00:03.202460,1,551.0


### Get ticks last count

In [ ]:
ticks = api.ticks(
    contract=api.Contracts.Stocks["2330"], 
    date="2020-03-04", 
    query_type=sj.constant.TicksQueryType.LastCount,
    last_cnt=5,  # 指定要抓最後5筆tick資料
)
ticks

Ticks(ts=[1583328289186000000, 1583328294207000000, 1583328299227000000, 1583328600000000000, 1583332200000000000], close=[320.0, 320.5, 320.0, 320.5, 320.5], volume=[11, 4, 3, 5571, 111], bid_price=[320.0, 320.0, 320.0, 320.0, 320.0], bid_volume=[374, 293, 366, 547, 547], ask_price=[320.5, 320.5, 320.5, 320.5, 320.5], ask_volume=[70, 134, 135, 59, 59], tick_type=[2, 1, 2, 1, 1])

In [ ]:
import pandas as pd
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df.head()

,close,ts,ask_price,tick_type,bid_volume,ask_volume,volume,bid_price
0,320.0,2020-03-04 13:24:49.186,320.5,2,374,70,11,320.0
1,320.5,2020-03-04 13:24:54.207,320.5,1,293,134,4,320.0
2,320.0,2020-03-04 13:24:59.227,320.5,2,366,135,3,320.0
3,320.5,2020-03-04 13:30:00.000,320.5,1,547,59,5571,320.0
4,320.5,2020-03-04 14:30:00.000,320.5,1,547,59,111,320.0


TicksQueryType，只有AllDay、RangeTime及LastCount這三種，使用方式就如同上面的這三個範例，但請注意這三種方式無法混用，因為api.ticks主要是依照`query_type`這個參數，去決定要抓哪些資料。

## KBar Data

In [ ]:
from shioaji.data import Kbars
Kbars??

Init signature:
Kbars(
    *,
    ts: List[int],
    Open: List[float],
    High: List[float],
    Low: List[float],
    Close: List[float],
    Volume: List[int],
    Amount: List[float],
) -> None
Source:        
class Kbars(BaseMapping):
    ts: typing.List[int]
    Open: typing.List[float]
    High: typing.List[float]
    Low: typing.List[float]
    Close: typing.List[float]
    Volume: typing.List[int]
    Amount: typing.List[float]

    def lazy_setter(self, **kwargs):
        [
            setattr(self, kwarg, value)
            for kwarg, value in kwargs.items()
            if hasattr(self, kwarg)
        ]
File:           c:\users\alan\anaconda3\envs\python3.7\lib\site-packages\shioaji\data.py
Type:           ModelMetaclass
Subclasses:     


In [ ]:
# 看起來Kbars產生的資料，其間隔時間最小單位是分鐘
kbars = api.kbars(api.Contracts.Stocks["2330"], start="2020-06-01", end="2020-07-01")
df = pd.DataFrame({**kbars}) # 先將Kbars物件轉換為Dict，再傳入DataFrame做轉換
df.ts = pd.to_datetime(df.ts) # 將原本的ts欄位中的資料，轉換為DateTime格式並回存
df

,Amount,High,Volume,Open,ts,Low,Close
0,936947000.0,294.0,3187,294.0,2020-06-01 09:01:00,293.5,294.0
1,59079000.0,294.0,201,293.5,2020-06-01 09:02:00,293.5,293.5
2,78422000.0,294.0,267,293.5,2020-06-01 09:03:00,293.5,294.0
3,183489000.0,294.5,624,294.0,2020-06-01 09:04:00,293.5,294.5
4,123194000.0,294.5,419,294.0,2020-06-01 09:05:00,293.5,294.0
...,...,...,...,...,...,...,...
5581,84010000.0,317.5,265,317.5,2020-07-01 13:22:00,317.0,317.0
5582,42424000.0,317.0,134,316.5,2020-07-01 13:23:00,316.5,317.0
5583,42429000.0,317.0,134,316.5,2020-07-01 13:24:00,316.5,316.5
5584,66547000.0,317.0,210,316.5,2020-07-01 13:25:00,316.5,316.5


> 從上面的輸出結果，可以看到kbars的資料時間都是一分鐘一筆資料，也就是對應K線的最小單位-1分鐘圖(1分K)。

api.kbars回傳的資料為shioaji.data.Kbars物件屬性說明如下：

|屬性|值|說明|
|:----|:----|:----|
|ts|[1631869260000000000]|時間戳|
|Open|[600.0]|開盤|
|High|[601.0]|最高|
|Low|[600.0]|最低|
|Close|[601.0]|收盤|
|Volume|[1316]|成交量|
|Amount|[789625000.0]|成交金額|

---
### Prev: [Market Data / Streaming Market Data](shioaji-market_data-streaming_market_data.ipynb) Next: [Market Data / Snapshot](shioaji-market_data-snapshot.ipynb)